This is to iterate over clean and noisy clips to compare them and find the biggest contrast for training

In [ ]:
import numpy as np
import librosa
import librosa.display
from IPython.display import Audio, display
import numpy as np

import os
#import time
import cmath

In [ ]:
# Get shared constants and functions
%run "NN Audio Core.py"

In [ ]:
# Share these?

def display_fft(ft):
    librosa.display.specshow(librosa.amplitude_to_db(np.abs(ft), ref=np.max), y_axis='cqt_hz', x_axis='time')

def draw(wav):
    fft = get_ft(wav)
    display_fft(fft)
    return fft

In [ ]:
# some test data to hack around with
file = "p232_001.wav"
clean_file = "Assets\\DataShareArchive\\Test\\Clean\\" + file
noisy_file = "Assets\\DataShareArchive\\Test\\Noisy\\" + file
wav, rate = librosa.core.load(clean_file)
noisy_wav, rate = librosa.core.load(noisy_file)

clean_ft = get_ft(wav)
noisy_ft = get_ft(noisy_wav)

print("Clean")
print(clean_ft.shape)
display_fft(clean_ft)

print(diff_ft(clean_ft,noisy_ft))
Audio(wav, rate=rate)

In [ ]:
print("Noisy")
print(noisy_ft.shape)
display_fft(noisy_ft)
Audio(noisy_wav, rate=rate)

In [ ]:
# Iterate over clean & noisy folders to create frames and targets
def compare_files(wav_root, min_diff = 150):
    clean_dir = wav_root + "\\Clean\\"
    noisy_dir = wav_root + "\\Noisy\\"
    file_list = os.listdir(clean_dir)
    file_index = 0
    while (file_index < len(file_list)) :
        file = file_list[file_index]
        noisy_ft = get_ft_from_file(noisy_dir + file)
        clean_ft = get_ft_from_file(clean_dir + file)
        diff = diff_ft(clean_ft, noisy_ft)
        if (diff > min_diff):
            print(file, diff)
        file_index += 1
    return 


In [ ]:
# small test data "Assets\\DataShareArchive\\Test"
# 28K "F:\\Audiodata\\Train28Spk"
compare_files("Assets\\DataShareArchive\\Test", min_diff=0)